In [3]:
# This code works for fixing unprocessable timetable csv files into a csv file that Google Calendar can read. I've not checked if it works with other calendar providers.
import pandas as pd
from datetime import datetime

# Load the raw timetable
df = pd.read_csv('Raw.csv')

# Columns we need (adjust if your column order differs slightly)
# Based on your file: A=Day, B=StartTime, C=FinishTime, F=Dates, H=Module name, P=Site name, R=Room name, Y=Event Type, Z=Module delivery type
df.columns = df.columns.str.strip()  # Remove any extra spaces

events = []

for _, row in df.iterrows():
    day = row['Day']
    start_time = str(row['StartTime']).strip()
    finish_time = str(row['FinishTime']).strip()
    dates_cell = str(row['Dates']).strip()
    module_name = row['Module name']
    site_name = row['Site name']
    room_name = row['Room name']
    event_type = row['Event Type']
    delivery_type = row['Module delivery type']

    # Parse dates: handle both "DD/MM/YYYY,DD/MM/YYYY" and "YYYY-MM-DD HH:MM:SS"
    date_list = []
    if ',' in dates_cell:
        # Comma-separated UK dates
        date_list = [d.strip() for d in dates_cell.split(',')]
    else:
        # Single datetime stamp or UK date
        date_list = [dates_cell]

    for date_str in date_list:
        if not date_str or date_str == 'nan':
            continue

        # Try parsing as DD/MM/YYYY
        try:
            event_date = datetime.strptime(date_str, '%d/%m/%Y')
        except:
            # Try parsing as YYYY-MM-DD HH:MM:SS
            try:
                event_date = datetime.strptime(date_str.split()[0], '%Y-%m-%d')
            except:
                print(f"Skipping unparseable date: {date_str}")
                continue

        # Format for Google Calendar CSV
        subject = f"{module_name} - {event_type} ({delivery_type})"
        start_date_str = event_date.strftime('%m/%d/%Y')  # MM/DD/YYYY for Google
        end_date_str = start_date_str  # Same day
        location = f"{site_name} - {room_name}"

        events.append({
            'Subject': subject,
            'Start Date': start_date_str,
            'Start Time': start_time,
            'End Date': end_date_str,
            'End Time': finish_time,
            'All Day Event': 'False',
            'Description': subject,
            'Location': location
        })

# Create DataFrame and export
export_df = pd.DataFrame(events)
export_df.to_csv('Google_Calendar_Import.csv', index=False)
print(f"✅ Created Google_Calendar_Import.csv with {len(export_df)} events")
print(f"First 5 events:\n{export_df.head()}")


✅ Created Google_Calendar_Import.csv with 82 events
First 5 events:
                                             Subject  Start Date Start Time  \
0  Insurance and Big Data - TT - Semester 2 on-ca...  02/23/2026      10:00   
1  Insurance and Big Data - TT - Semester 2 on-ca...  03/02/2026      10:00   
2  Insurance and Big Data - TT - Semester 2 on-ca...  03/09/2026      10:00   
3  Insurance and Big Data - TT - Semester 2 on-ca...  03/16/2026      10:00   
4  Insurance and Big Data - TT - Semester 2 on-ca...  03/23/2026      10:00   

     End Date End Time All Day Event  \
0  02/23/2026    12:00         False   
1  03/02/2026    12:00         False   
2  03/09/2026    12:00         False   
3  03/16/2026    12:00         False   
4  03/23/2026    12:00         False   

                                         Description        Location  
0  Insurance and Big Data - TT - Semester 2 on-ca...  ICMA - G07 PC*  
1  Insurance and Big Data - TT - Semester 2 on-ca...  ICMA - G07 PC*  
2  